# Aplica correlação para excluir variáveis correlacionadas (70%)

In [ ]:
import os
import pandas as pd
import numpy as np

def calculate_correlation_matrix(data, target_column, correlation_threshold=0.9):
    """
    Calcula a matriz de correlação e identifica variáveis altamente correlacionadas.

    Args:
        data (pd.DataFrame): Dataset contendo as variáveis.
        target_column (str): Nome da coluna alvo.
        correlation_threshold (float): Limite de correlação para exclusão de variáveis.

    Returns:
        list: Lista de variáveis a serem removidas.
    """
    correlation_matrix = data.drop(columns=[target_column]).corr()
    columns_to_remove = set()

    for i in range(len(correlation_matrix.columns)):
        for j in range(i):
            if abs(correlation_matrix.iloc[i, j]) > correlation_threshold:
                col_to_remove = correlation_matrix.columns[i]
                columns_to_remove.add(col_to_remove)
    
    return list(columns_to_remove)

def remove_highly_correlated_features(data, target_column, correlation_threshold=0.9):
    """
    Remove variáveis altamente correlacionadas com base em um limite.

    Args:
        data (pd.DataFrame): Dataset original.
        target_column (str): Nome da coluna alvo.
        correlation_threshold (float): Limite de correlação para exclusão de variáveis.

    Returns:
        pd.DataFrame: Dataset com variáveis correlacionadas removidas.
    """
    columns_to_remove = calculate_correlation_matrix(data, target_column, correlation_threshold)
    print(f"Variáveis removidas devido à alta correlação: {columns_to_remove}")
    return data.drop(columns=columns_to_remove)

def get_next_folder_name(output_root):
    """
    Calcula o próximo nome de pasta numerado com base nas pastas existentes no diretório de saída.

    Args:
        output_root (str): Diretório raiz para as pastas de saída.

    Returns:
        int: Próximo número disponível para a pasta.
    """
    existing_folders = [
        folder for folder in os.listdir(output_root) if os.path.isdir(os.path.join(output_root, folder))
    ]
    numbered_folders = [
        int(folder.split("-")[0]) for folder in existing_folders if folder.split("-")[0].isdigit()
    ]
    return max(numbered_folders, default=0) + 1

def process_dataset_in_folder(input_dir, output_dir, correlation_threshold=0.9):
    """
    Processa os datasets em uma pasta, remove variáveis correlacionadas e salva os resultados.

    Args:
        input_dir (str): Caminho do diretório de entrada.
        output_dir (str): Caminho do diretório de saída.
        correlation_threshold (float): Limite de correlação para exclusão de variáveis.
    """
    os.makedirs(output_dir, exist_ok=True)

    for filename in os.listdir(input_dir):
        if filename.endswith(".txt"):
            input_path = os.path.join(input_dir, filename)
            output_path = os.path.join(output_dir, filename)

            print(f"Processando arquivo: {input_path}")
            try:
                data = pd.read_csv(input_path, delimiter=",", skipinitialspace=True)
                target_column = data.columns[-1]

                # Remover variáveis altamente correlacionadas
                data_cleaned = remove_highly_correlated_features(data, target_column, correlation_threshold)

                # Reorganizar as colunas para manter a coluna alvo no final
                columns = [col for col in data_cleaned.columns if col != target_column] + [target_column]
                data_cleaned = data_cleaned[columns]

                data_cleaned.to_csv(output_path, sep=",", index=False)
                print(f"Arquivo processado e salvo em: {output_path}")

            except Exception as e:
                print(f"Erro ao processar {filename}: {e}")

def process_multiple_folders(input_root, output_root, correlation_threshold=0.9):
    """
    Processa múltiplas pastas dentro do diretório de entrada e salva os resultados diretamente
    no diretório de saída com nomes incrementados.

    Args:
        input_root (str): Diretório raiz contendo as pastas de entrada.
        output_root (str): Diretório raiz para salvar as pastas processadas.
        correlation_threshold (float): Limite de correlação para exclusão de variáveis.
    """
    next_folder_number = get_next_folder_name(output_root)

    for folder_name in os.listdir(input_root):
        input_folder_path = os.path.join(input_root, folder_name)

        if os.path.isdir(input_folder_path):  # Verifica se é uma pasta
            output_folder_name = f"{next_folder_number}-saida"
            output_folder_path = os.path.join(output_root, output_folder_name)

            print(f"\nProcessando pasta {folder_name} e salvando em: {output_folder_path}")
            process_dataset_in_folder(input_folder_path, output_folder_path, correlation_threshold)

            next_folder_number += 1

# Diretórios de entrada e saída
input_root_directory = r"C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\algorithms\abordagens\output"
output_root_directory = r"C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\algorithms\abordagens\output2"

# Processar arquivos removendo variáveis correlacionadas
process_multiple_folders(input_root_directory, output_root_directory, correlation_threshold=0.7)



Processando pasta 0-saida e salvando em: C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\algorithms\abordagens\output2\27-saida
Processando arquivo: C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\algorithms\abordagens\output\0-saida\cocomo81.txt
Variáveis removidas devido à alta correlação: ['time', 'lexp']
Arquivo processado e salvo em: C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\algorithms\abordagens\output2\27-saida\cocomo81.txt
Processando arquivo: C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\algorithms\abordagens\output\0-saida\encoded_china.txt
Variáveis removidas devido à alta correlação: ['Input', 'Output', 'Resource_2', 'Added', 'NPDR_AFP', 'PDR_UFP', 'NPDU_UFP', 'File']
Arquivo processado e salvo em: C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\algorithms\abordagens\output

: 